# Projeto 1 - Ciência dos Dados

Nome: Lourenço Meirelles

Nome: Lucca Lima

Nome: Pedro Drumond

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [54]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [55]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Pedro Drumond\Documents\Insper 2o semestre\CDados\GitHub_CD\Projeto_1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [56]:
filename = 'Primevideo.xlsx'

In [57]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificacao
0,"""eu sou mais eu"" não tem roteiro criativo, par...",2
1,#nineperfectstrangers \nsérie muito boa no @pr...,3
2,#primevideo quero uma sequência pra ontem de ...,2
3,@camila_cabello @cinderella @primevideo amei p...,3
4,@camila_cabello @cinderella @primevideo bravo!👏,3


In [58]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,@janainadobrasil esse daqui é a melhor opção p...,1
1,"@camila_cabello @primevideo ficou lindo, te amo",2
2,@camila_cabello @cinderella @primevideo linda!...,3
3,@camila_cabello @cinderella @primevideo ta lin...,3
4,@camilizecabello @mannytsakanika @cinderella @...,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O Amazon Prime Video é uma plataforma de streaming semelhante à Netflix, na qual você pode acessar um catálogo de filmes, séries, etc... além de disponibilizar outras coisas como o Amazon Prime Music e entre outros.

Para classificar, consideramos mensagens "automáticas", prontas, divulgando links como 'muito irrelevantes'. Mensagens aleatórias e não relacionadas ao assunto como 'irrelevantes'. Mensagens que elogiassem a plataforma como 'relevantes' e mensagens que elogiassem muito a plataforma como 'muito relevantes'.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [59]:
def cleanup(text):
    #import string
    punctuation = '[\n!-.:@?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    #MELHORAR
    
#tirar '@'
#talvez 'e', 'que'

In [60]:
#PRECISAMOS DE:

#P(mensagens relevantes | classificadas como relevantes)
#P(mensagens irrelevantes | classificadas como relevantes)
#P(mensagens irrelevantes | classificadas como irrelevantes)
#P(mensagens relevantes | classificadas como irrelevantes)
#Acurácia ?

In [61]:
#DAR UMA OLHADA EM PANDAS
# Carrega o arquivo de texto
# Como ele possui caracteres com acentos, precisamos carregar utilizando o encoding="utf8".
primevideo = pd.read_excel('Primevideo.xlsx')
primevideo.Classificacao = primevideo.Classificacao.astype('int')
primevideo
primevideo_teste = pd.read_excel('Primevideo.xlsx', sheet_name=1)
primevideo_teste

,Teste,Classificação
0,@janainadobrasil esse daqui é a melhor opção p...,1
1,"@camila_cabello @primevideo ficou lindo, te amo",2
2,@camila_cabello @cinderella @primevideo linda!...,3
3,@camila_cabello @cinderella @primevideo ta lin...,3
4,@camilizecabello @mannytsakanika @cinderella @...,2
...,...,...
294,lançamentos do prime video em setembro de 2021...,0
295,"já é tarde demais para dizer que cinderella, d...",1
296,@teamyoutube minha tv (sony) e meu celular (sa...,1
297,@anyborgesreal @dougfrn baseado no caso ritcho...,0


In [62]:
#Criar um DF (DataFrame) para cada classificação

#Filtra as linhas, separando cada clasificação em um DF sem ter aplicado o 'CleanUp'
filtra_linhas = primevideo.Classificacao == 0
primevideo_classfic_0_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

filtra_linhas = primevideo.Classificacao == 1
primevideo_classfic_1_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 2
primevideo_classfic_2_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 3
primevideo_classfic_3_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

print(primevideo_classfic_0_raw)

0      @artofesthetic baseado no caso ritchofen, dois...
1      @laressa baseado no caso ritchofen, dois filme...
2      @popzonebr baseado no caso ritchofen, dois fil...
3      @bbb @gshow baseado no caso ritchofen, dois fi...
4      @flamengo @davidluiz_4 baseado no caso ritchof...
                             ...                        
142    @paulaamorim baseado no caso ritchofen, dois f...
143    @eulaurakeller @valescaoficial baseado no caso...
144    @sepcarloslima @acordadebora um crime real, do...
145    @acordadebora um crime real, dois filmes e dua...
146    @rolealeatorio baseado no caso ritchofen, dois...
Name: Treinamento, Length: 147, dtype: object


In [63]:
#CLEANUP - vai precisar do for
#cleanup(primevideo_classfic_0_raw[0].lower()).split()

In [64]:
a = [1, 3]
b = [1, 4]
print(a+b)
c = b.append(a)
print(c)

[1, 3, 1, 4]
None


In [65]:
#For, criando as listas limpas
lista_primevideo_classfic_0 = []
lista_primevideo_classfic_1 = []
lista_primevideo_classfic_2 = []
lista_primevideo_classfic_3 = []

lista_com_todos_tuites_limpos = []

#Cria a lista lista_primevideo_classfic que possui  as classificações de cada categoria
lista_tuites = []
for celula in primevideo_classfic_0_raw:
    
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_0 += celula_limpa.split()
    #print(lista_primevideo_classfic_0)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_1_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_1 += celula_limpa.split()
    #print(lista_primevideo_classfic_1)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_2_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_2 += celula_limpa.split()
    #print(lista_primevideo_classfic_2)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_3 += celula_limpa.split()
    #print(lista_primevideo_classfic_3)
    #lista_tuites.append(celula_limpa.split())
    
'''
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower()).split()
    lista_primevideo_classfic_3 += celula_limpa
    print(lista_primevideo_classfic_3)
'''



'\nfor celula in primevideo_classfic_3_raw:\n    celula_limpa = cleanup(celula.lower()).split()\n    lista_primevideo_classfic_3 += celula_limpa\n    print(lista_primevideo_classfic_3)\n'

In [66]:
# cria a lista tuite a partir da planilha Teste
for tuite in primevideo_teste['Teste']:
    celula_limpa = cleanup(tuite.lower())
    lista_tuites.append(celula_limpa.split())

In [67]:
#Convertendo as tabelas para pd.Series

classfic_0 = pd.Series(lista_primevideo_classfic_0)
print(classfic_0.value_counts())
print('##############################################')
classfic_1 = pd.Series(lista_primevideo_classfic_1)
print(classfic_1.value_counts())
print('##############################################')
classfic_2 = pd.Series(lista_primevideo_classfic_2)
print(classfic_2.value_counts())
print('##############################################')
classfic_3 = pd.Series(lista_primevideo_classfic_3)
classfic_3.value_counts()


e                        115
que                       96
matou                     82
pais                      82
prime                     79
                        ... 
esteja                     1
viajantes                  1
https//tco/bc8ukinfuk      1
exibição                   1
incríveis                  1
Length: 853, dtype: int64
##############################################
prime         96
o             70
primevideo    69
e             68
a             65
              ..
videose        1
atras          1
possui         1
cruella        1
legendado      1
Length: 1295, dtype: int64
##############################################
primevideo               72
camila_cabello           51
de                       34
e                        34
a                        33
                         ..
https//tco/u5eteau6qy     1
curta                     1
br                        1
dias                      1
espera                    1
Length: 616, dtype: int64
#############

a               41
primevideo      33
o               31
e               31
prime           28
                ..
kaykaycannon     1
conto            1
entendam         1
praticamente     1
entender         1
Length: 569, dtype: int64

In [68]:
#Tabelas de contagem de ocorrencias em cada tabela, usando o value_counts()
tabela_classific_0 = classfic_0.value_counts()
tabela_classific_1 = classfic_1.value_counts()
tabela_classific_2 = classfic_2.value_counts()
tabela_classific_3 = classfic_3.value_counts()

#Juntando todos os tuítes
todos_tuites = lista_primevideo_classfic_0 + lista_primevideo_classfic_1 + lista_primevideo_classfic_2 + lista_primevideo_classfic_3
#Criando tabela dos tuítes usando o pd.Series
todos_tuites_lista = pd.Series(todos_tuites)
tabela_todos_tuites = todos_tuites_lista.value_counts()
tabela_todos_tuites
#tabela_classific_0

#ACHO QUE VAMOS TER QUE USAR TABELA RELATIVA PORQUE TEMOS MAIS AVALIAÇÕES EM UNS DO QUE OUTROS, VER ISSO!

e             248
prime         225
primevideo    208
a             207
que           198
             ... 
sampson_iv      1
infos           1
cavaleiro       1
kay             1
irá             1
Length: 2552, dtype: int64

In [69]:
tabela_0 = classfic_0.value_counts()
tabela_0

e                        115
que                       96
matou                     82
pais                      82
prime                     79
                        ... 
esteja                     1
viajantes                  1
https//tco/bc8ukinfuk      1
exibição                   1
incríveis                  1
Length: 853, dtype: int64

In [70]:
tabela_1 = classfic_1.value_counts()
tabela_1

prime         96
o             70
primevideo    69
e             68
a             65
              ..
videose        1
atras          1
possui         1
cruella        1
legendado      1
Length: 1295, dtype: int64

In [71]:
tabela_2 = classfic_2.value_counts()
tabela_2

primevideo               72
camila_cabello           51
de                       34
e                        34
a                        33
                         ..
https//tco/u5eteau6qy     1
curta                     1
br                        1
dias                      1
espera                    1
Length: 616, dtype: int64

In [72]:
tabela_3 = classfic_3.value_counts()
tabela_3

a               41
primevideo      33
o               31
e               31
prime           28
                ..
kaykaycannon     1
conto            1
entendam         1
praticamente     1
entender         1
Length: 569, dtype: int64

In [73]:
tuite = '@rolealeatorio baseado no caso ritchofen, dois filmes e duas versões: a menina que matou os pais e o menino que matou meus pais'
tuite = cleanup(tuite.lower())
tuite.split()

['rolealeatorio',
 'baseado',
 'no',
 'caso',
 'ritchofen',
 'dois',
 'filmes',
 'e',
 'duas',
 'versões',
 'a',
 'menina',
 'que',
 'matou',
 'os',
 'pais',
 'e',
 'o',
 'menino',
 'que',
 'matou',
 'meus',
 'pais']

In [75]:
lista_tuites

[['janainadobrasil',
  'esse',
  'daqui',
  'é',
  'a',
  'melhor',
  'opção',
  'pra',
  'completar',
  'o',
  'fim',
  'de',
  'semana',
  'https//tco/0iqdwkgap8'],
 ['camila_cabello', 'primevideo', 'ficou', 'lindo', 'te', 'amo'],
 ['camila_cabello', 'cinderella', 'primevideo', 'linda', 'eu', 'amei🥰🥰🥰'],
 ['camila_cabello',
  'cinderella',
  'primevideo',
  'ta',
  'lindo',
  'amor',
  'amei',
  'tudo'],
 ['camilizecabello',
  'mannytsakanika',
  'cinderella',
  'primevideo',
  'kkkkkkk',
  'venceu',
  'amg'],
 ['camila_cabello',
  'primevideo',
  'tô',
  'tão',
  'feliz',
  'por',
  'você',
  '🥺❤️❤️'],
 ['filmaço',
  'no',
  'prime',
  'video',
  'primevideobr',
  'mid90s',
  'https//tco/h3urdlzdcb'],
 ['cinderella', 'primevideo', 'assisti', 'várias', 'vezes', 'e', 'amei😍😍😍😭😭😭'],
 ['filmaço', 'clássicohttps//tco/xoimn2bueg'],
 ['camila_cabello', 'primevideo', 'te', 'amo', 'te', 'amo', 'te', 'amo'],
 ['camila_cabello', 'cinderella', 'primevideo', 'eu', 'te', 'amoo'],
 ['cinderela',
 

In [82]:
classificacao_final = []
for tuite in lista_tuites:
    prob_0 = 1
    prob_1 = 1
    prob_2 = 1
    prob_3 = 1
    
    #dentro desse FOR teremos: -probabilidade das palavras, -descobre todas as probabilidades, -aplica La Place, -descobre qual a classificacao
    for palavra in tuite:
        
        #LA PLACE
        if palavra not in tabela_0:
            prob_0 *= 1/(len(classfic_0) + len(tabela_todos_tuites)) 
            #          /todas as palavras na categoria + todas as palavras de todas as categorias (sem repetição)
            #          acho que não está pegando todas as palavras ta tabela 0           
        elif palavra in tabela_0:
            prob_0 *= (tabela_0[palavra] + 1)/(len(classfic_0) + len(tabela_todos_tuites))
        
        if palavra not in tabela_1:
            prob_1 *= 1/(len(classfic_1) + len(tabela_todos_tuites)) #talvez seja a tabela errada, rever porque isso é do 'Treinamento'
                          #quantida de palavras na categoria 1 + quantidade de palavras no total
        elif palavra in tabela_1:
            prob_1 *= (tabela_1[palavra] + 1)/(len(classfic_1) + len(tabela_todos_tuites))
        
        if palavra not in tabela_2:
            prob_2 *= 1/(len(classfic_2) + len(tabela_todos_tuites)) #talvez seja a tabela errada, rever porque isso é do 'Treinamento'
        elif palavra in tabela_2:
            prob_2 *= (tabela_2[palavra] + 1)/(len(classfic_2) + len(tabela_todos_tuites))
        
        if palavra not in tabela_3:
            prob_3 *= 1/(len(classfic_3) + len(tabela_todos_tuites)) #talvez seja a tabela errada, rever porque isso é do 'Treinamento'
        elif palavra in tabela_3:
            prob_3 *= (tabela_3[palavra] + 1)/(len(classfic_3) + len(tabela_todos_tuites))
        

        #COMPARAÇÃO ENTRE OS RESULTADOS DE LA PLACE PARA CADA CATEGORIA
        if prob_0 > prob_1 and prob_0 > prob_2 and prob_0 > prob_3:
            #print(prob_0)
            classificacao_final.append(0)
            #adicionar na coluna no excel
            
        elif prob_1 > prob_0 and prob_1 > prob_2 and prob_1 > prob_3:
            #print(prob_1)
            classificacao_final.append(1)
            
        elif prob_2 > prob_0 and prob_2 > prob_1 and prob_2 > prob_3:
            #print(prob_2)
            classificacao_final.append(2)
            
        elif prob_3 > prob_0 and prob_3 > prob_1 and prob_3 > prob_2:
            #print(prob_3)
            classificacao_final.append(3)
        
        # denominador: ocorrencia da palavra na categoria + 1 / todas as palavras na categoria + todas as palavras de todas as categorias (sem repetição)
        # prob_A_dado_B = prob_B_dado_A * prob_A
        
        '''
        prob_palavras_dado_0 *= tabela_0[palavra]
        prob_1 *= tabela_1[palavra]
        prob_2 *= tabela_2[palavra]
        prob_3 *= tabela_3[palavra]
        '''
        
        #Chama função que faz o calculo de P_classificação_dado_tuite:
        
        #prob_0_dado_tuite = prob_0 * ( len(tabela_0) / len(tabela_todos_tuites )
        #                                                        esse len() está errado
        
        '''
        prob_0_dado_tuite = calcula_prob_classific_dado_tuite(prob_0, len(tabela_0), len(tabela_todos_tuites))
        prob_1_dado_tuite = calcula_prob_classific_dado_tuite(prob_1, len(tabela_1), len(tabela_todos_tuites))
        prob_2_dado_tuite = calcula_prob_classific_dado_tuite(prob_2, len(tabela_2), len(tabela_todos_tuites))
        prob_3_dado_tuite = calcula_prob_classific_dado_tuite(prob_3, len(tabela_3), len(tabela_todos_tuites))
        '''


#FAZER ALGUMA COISA COM O 'classificacao_final' QUE É UMA TABELA COM AS CLASSIFICAÇÕES

    
        

In [96]:
#VERIFICANDO A PERFORMANCE

classificacao_final
classificacao_final_pd = pd.Series(classificacao_final)
primevideo_teste['Resultado Classificador'] = classificacao_final_pd
primevideo_teste.tail(20)

pd.crosstab(primevideo_teste['Classificação'], primevideo_teste['Resultado Classificador'], normalize=True).round(4)*100 #linha e coluna

Resultado Classificador,0,1,2,3
Classificação,,,,
0,1.00,4.68,16.72,4.68
1,1.34,8.36,25.75,7.69
2,0.00,1.00,8.36,8.03
3,0.33,0.33,6.02,5.69


In [ ]:
'''FALTA FAZER:
    
    URGENTE!
    - Verificar se o uso de La Place está correto!!!
    
    - Verificar se a verificação da performance está correta
    - Produzir o texto com análise crítica e conclusão
    
    CONCEITO A:
    - Melhorar o cleanup()
    - Corrigir espaçamento nos tuítes
    - Explicar porque não podemos usar nosso classificador para gerar mais amostras de treinamento
    - ? Difirentes cenários para Naive Bayes fora do contexto do projeto
    - Sugerir melhorias reais 
    - 
'''

In [77]:
'''#                                   
def calcula_prob_classific_dado_tuite(prob_classific, tamanho_tabela_categoria, tamanho_tabela_todos_tuites_repeticoes):
    
    prob_classific_dado_tuite = prob_classific * (tamanho_tabela_categoria / tamanho_tabela_todos_tuites_repeticoes)
    return prob_classific_dado_tuite'''

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**